In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import sklearn

In [2]:
week = "/home/lucien/Bureau/projet_ml_finance/datahub/wti-week_csv.csv"
daily = "/home/lucien/Bureau/projet_ml_finance/datahub/wti-daily_csv.csv"

data_week = pd.read_csv(week, delimiter = ',')
data_daily = pd.read_csv(daily, delimiter = ',')

Preprocessing des données: format datetime et traitement des données manquantes

In [3]:
#format datime
data_week['Date'] = pd.to_datetime(data_week['Date'])

#on complète les valeurs de Price null avec la méthode forward fill

data_week['Price'].fillna(method='ffill', inplace=True)

/tmp/ipykernel_4602/779900049.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_week['Price'].fillna(method='ffill', inplace=True)


In [4]:
#split train et valid set

from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test (par exemple, 80% - 20%)
train_size = 0.8  # Taille de l'ensemble d'entraînement
train_data, test_data = train_test_split(data_daily, train_size=train_size, shuffle=False)


LET'S SELECTE THE HYPERPARAMETERS P, D AND Q OF THE ARIMA MODEL .

AIC = mesure statistique type grid search qui prend en compte la complexité du modèle.
For that we compute AIC (AIkake information criterion) for every sarimax. AIC = 2*(number of parameter - log(MLE for this model)). We want the min of AIC. 

In [5]:
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore") # specify to ignore warning messages

# Generate a grid of p, d, q values
p = d = q = range(0, 3)  # Try values from 0 to 2 for p, d, q, ce sont les intervalles dans lesquels varient les hyperparamètres
pdq = list(itertools.product(p, d, q))

# Perform grid search to find the best parameters, without cross validation because results.aic is done on empirical risk
subset_data = train_data['Price'].copy()
best_aic = np.inf
best_params = None
for param in pdq:
    model = sm.tsa.ARIMA(subset_data, order=param)
    results = model.fit()
    if results.aic < best_aic:
        best_aic = results.aic
        best_params = param
    

print(f"Best AIC: {best_aic}")
print(f"Best Parameters: {best_params}")


Best AIC: 20969.946282702287
Best Parameters: (2, 1, 2)


In [6]:
#we train the best model
model = sm.tsa.ARIMA(train_data['Price'], order=best_params)
results = model.fit()

#we evaluate the model on test_data['Price'] with the MAPE measure

test_data_ = test_data['Price'].copy()
def mean_absolute_percentage_error(y_true, y_pred):
    s = 0
    for i in range(len(y_true)):
        s += np.abs((y_true[i] - y_pred[i]) / y_true[i])
    s = s/5
    return s * 100

# Nombre de prédictions à effectuer pour chaque point dans l'ensemble de test
num_predictions = 5

# Liste pour stocker les résultats de la précision de prédiction (MAPE)
mape_results = []

# Boucle sur l'ensemble de test
for i in range(len(test_data_) - num_predictions):
    
    # Select the next 'num_predictions' points for prediction
    test_subset = test_data[i : i + num_predictions]
    # Make predictions for the next 'num_predictions' points
    forecast = results.forecast(steps=num_predictions)
    # Calculate MAPE for these predictions
    
    mape = mean_absolute_percentage_error(list(test_subset['Price']), list(forecast))
    mape_results.append(mape)
# Calculate the average MAPE across all predictions
average_mape = np.mean(mape_results)
print(f"Average MAPE: {average_mape}")


Average MAPE: 52.63648939468439
